# Generate a Project Template

## Imports

In [8]:
from typing import Dict, List

import subprocess
import json
import re
from pathlib import Path
from dataclasses import dataclass, InitVar
from pprint import pprint

import pandas as pd


In [9]:
from env_tools import AnacondaException, AbortedCmdException


In [10]:
from env_tools import console_command, activate_environment


In [25]:
from env_tools import save_env_specs, list_environments

from env_tools import create_environment, remove_environment

from env_tools import install_packages, pip_install_packages


In [12]:
# %% Initialize logging
import logging  # pylint: disable=wrong-import-position wrong-import-order
logging.basicConfig(level=logging.INFO,
                    format='%(levelname)-8s %(message)s')
logger = logging.getLogger(__name__)
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)


## Utility Functions

In [13]:
def add_folder(base: Path, name: str)->Path:
    '''Make a subdirectory under base.

    Generates a path to the subdirectory.
    If it doesn't exist create it.

    Args:
        base (Path): The top folder (parent to the subdirectory).
        name (str): The name of the subdirectory.

    Returns:
        Path: The path to the subdirectory.
    '''
    sub_folder = base / name
    if not sub_folder.exists():
        sub_folder.mkdir()
    return sub_folder


## Project Parameters

### Project Name and Python Version

In [ ]:
new_env = 'Todoist'
python_version = '3.11'


### Packages to include in the Conda Environment

In [ ]:
primary_package_list = [
    'pysimplegui',
    'pandas',
    'xlwings',
    ]
tools_package_list = [
    'spyder',
    'jupyterlab',
    'ipykernel',
    'pylint',
    ]
package_list = primary_package_list + tools_package_list
packages = ' '.join(package_list)


In [ ]:
install_xlwings = True
install_jupyter = True


## Package Creation

### Create the new environment

In [14]:
new_env = 'TestProject'
python_version = '3.9'


In [15]:
create_output = create_environment(new_env, python_version)


INFO     Creating environment for TestProject


#### Sample of `create` output

In [16]:
print(list(create_output.keys()))
print(create_output['success'])
print(create_output['prefix'])
print(create_output['actions'].keys())
if len(create_output['actions']['LINK']) > 0:
    pprint(create_output['actions']['LINK'][0])


['actions', 'prefix', 'success']
True
C:\Users\smoke\.conda\envs\TestProject
dict_keys(['FETCH', 'LINK', 'PREFIX'])
{'base_url': 'https://conda.anaconda.org/conda-forge',
 'build_number': 0,
 'build_string': 'h56e8100_0',
 'channel': 'conda-forge',
 'dist_name': 'ca-certificates-2023.7.22-h56e8100_0',
 'name': 'ca-certificates',
 'platform': 'win-64',
 'version': '2023.7.22'}


In [17]:
# Identify the Conda Environment path
env_path = Path(create_output['prefix'])


### Install the Packages

#### Packages to include in the Conda Environment

In [18]:
primary_package_list = [
    'pysimplegui',
    'pandas',
    'xlwings',
    ]
tools_package_list = [
    'spyder',
    'jupyterlab',
    'ipykernel',
    'pylint',
    ]
package_list = primary_package_list + tools_package_list


In [19]:
install_output = install_packages(new_env, package_list)


#### Sample of `install` output

In [20]:
print(list(install_output.keys()))
print(install_output['success'])
print(install_output['prefix'])
print(install_output['actions'].keys())
if len(install_output['actions']['LINK']) > 0:
    pprint(install_output['actions']['LINK'][0])


['actions', 'prefix', 'success']
True
C:\Users\smoke\.conda\envs\TestProject
dict_keys(['FETCH', 'LINK', 'PREFIX'])
{'base_url': 'https://conda.anaconda.org/conda-forge',
 'build_number': 50496,
 'build_string': 'h57928b3_50496',
 'channel': 'conda-forge',
 'dist_name': 'intel-openmp-2023.2.0-h57928b3_50496',
 'name': 'intel-openmp',
 'platform': 'win-64',
 'version': '2023.2.0'}


### Install packages using `pip`

In [21]:
# packages that are not available from Conda
pip_package_list = [
    'todoist-api-python',
    'units'
    ]


In [34]:
def pip_install_packages(env_ref: EnvRef, pip_package_list: List[str])->Dict[str,str]:
    '''Use pip to install packages in an Anaconda environment

    This is intended to be used for installing packages that that cannot be
    found in one of the Anaconda channels.

    Args:
        env_ref (EnvRef): A reference to the Conda environment either by it's
            name or by the path to the environment.
        package_list (List[str]): A list of pip "requirement specifiers"
            (typically package names) to install in the Conda environment.  See
            [Requirement Specifiers](https://pip.pypa.io/en/stable/reference/requirement-specifiers/)
            for more details.
    Returns:
        Dict[str,str]: Log output, as a dictionary of dictionaries, generated
            when installing the packages.
    '''
    env_name = set_env_ref(env_ref)[0]
    activate_cmd = activate_environment(env_name)

    install_logs = []
    for pkg_req in pip_package_list:
        install_cmd = activate_cmd + f'pip install {pkg_req} --quiet --report -'
        err_msg = f'Unable to install {pkg_req} in "{env_name}"!'
        pip_output = console_command(install_cmd, AnacondaException, err_msg)
        install_log_dict = json.loads(pip_output)
        install_logs.append(install_log_dict)

    return install_logs


In [31]:
from env_tools import set_env_ref


In [33]:
set_env_ref(env_path)[0]


'TestProject'

In [35]:
pip_install_output = pip_install_packages(env_path, pip_package_list)


In [36]:
pprint(pip_install_output)


[{'environment': {'implementation_name': 'cpython',
                  'implementation_version': '3.9.18',
                  'os_name': 'nt',
                  'platform_machine': 'AMD64',
                  'platform_python_implementation': 'CPython',
                  'platform_release': '10',
                  'platform_system': 'Windows',
                  'platform_version': '10.0.19045',
                  'python_full_version': '3.9.18',
                  'python_version': '3.9',
                  'sys_platform': 'win32'},
  'install': [{'download_info': {'archive_info': {'hash': 'sha256=7d182ff9d663dddae445ad118cb56ad8596e70a2f0f38bd4ebfa40bbba60fb0a',
                                                  'hashes': {'sha256': '7d182ff9d663dddae445ad118cb56ad8596e70a2f0f38bd4ebfa40bbba60fb0a'}},
                                 'url': 'https://files.pythonhosted.org/packages/d8/69/60e613fcd8dea18105f653afcbbb48fb9a13cedc2403ab04f4f863b9dc9e/todoist_api_python-2.1.3-py3-none-any.whl'},


### Delete the Conda Environment

In [ ]:
#delete_output = remove_environment(env_path)


#### Sample of `delete` output

In [ ]:
#pprint(delete_output)


# Done To Here

In [23]:
list_environments()


[('base', WindowsPath('C:/ProgramData/Anaconda3')),
 ('Graphs', WindowsPath('C:/ProgramData/Anaconda3/envs/Graphs')),
 ('C2', WindowsPath('C:/Users/smoke/.conda/envs/C2')),
 ('EDW_QA', WindowsPath('C:/Users/smoke/.conda/envs/EDW_QA')),
 ('FormExtraction', WindowsPath('C:/Users/smoke/.conda/envs/FormExtraction')),
 ('McMedHack', WindowsPath('C:/Users/smoke/.conda/envs/McMedHack')),
 ('Standard', WindowsPath('C:/Users/smoke/.conda/envs/Standard')),
 ('TestPathDev', WindowsPath('C:/Users/smoke/.conda/envs/TestPathDev')),
 ('TestProject', WindowsPath('C:/Users/smoke/.conda/envs/TestProject')),
 ('sectionary', WindowsPath('C:/Users/smoke/.conda/envs/sectionary')),
 ('sectionaryDev', WindowsPath('C:/Users/smoke/.conda/envs/sectionaryDev')),
 ('variandb_relations',
  WindowsPath('C:/Users/smoke/.conda/envs/variandb_relations'))]

In [37]:
from env_tools import EnvRef
from env_tools import set_env_ref


## Project folder creation and search path management

## Specify project related paths
- The top level project path should already exist and should be given the tag 'WORKSPACE'
- `folder_path` must be specified for 'WORKSPACE'
- An 'ENVIRONMENT' must be defined.
- There shouldn't be duplicate tags.
- If `tag` is not given `folder_name.upper()` will be used as the `tag`
- When `folder_path` is given, the `folder_name` is ignored.  
- If `folder_path` is not given, a folder with the name `folder_name` will be created directly under `base_path`.  
- If `add_to_path` is True *(the default)*, the folder path will be added to the python search path.

#### Example folders:

| folder_name | folder_path                              | tag         | add_to_path |
|-------------|------------------------------------------|-------------|-------------|
| Todoist     | ~\Python\Projects\ToDoIst Projects\Turkey| WORKSPACE   | True        |
| environment | None                                     | ENVIRONMENT | False       |
| src         | None                                     | SOURCE      | True        |
| Old files   | None                                     | IGNORE      | False       |
| data        | None                                     | DATA        | True        |
| reference   | None                                     | REFERENCE   | False       |
| sectionary  | ..\\..\sectionary package\src'           | PACKAGE     | True        |

### Primary project path

In [ ]:
project_folder = Path(r"D:\OneDrive - Queen's University\Python\Projects\ToDoIst Projects\Turkey")
project_drive = project_folder.drive


### Additional package path

In [ ]:
sectionary_path = project_folder / '..' / '..' / 'sectionary package' / 'src'
sectionary_path = sectionary_path.resolve()


#### Folder Parameters

In [ ]:
folder_list = [FolderItem(new_env,       tag='WORKSPACE',                      folder_path=project_folder),
               FolderItem('environment', tag='ENVIRONMENT', add_to_path=False,   base_path=project_folder),
               FolderItem('src',         tag='SOURCE',                           base_path=project_folder),
               FolderItem('data',        tag='DATA',                             base_path=project_folder),
               FolderItem('reference',   tag='REFERENCE',   add_to_path=False,   base_path=project_folder),
               FolderItem('sectionary',                                        folder_path=sectionary_path)]

folders = pd.DataFrame(folder_list)

folders


### Class to hold information on folders related to the package.

In [ ]:
@dataclass
class FolderItem:
    '''Manage folders related to the package.'''
    folder_name: str
    tag: str = ''
    add_to_path: bool = True
    folder_path: Path = None
    base_path: InitVar[Path] = None

    def __post_init__(self, base_path: Path):
        '''Manage defaults for folder settings.

        If not defined, base_path defaults to the current working
        directory.  If it doesn't exists the folder is created.

        If tag is not given use uppercase of folder name for tag.
        '''
        if not base_path:
            base_path = Path.cwd()
        self.set_path(base_path)
        self.set_tag()

    def set_path(self, base_path: Path):
        '''If missing, Set the folder_path to a subdirectory of base_path.

        Generates a path to the subdirectory.
        If it doesn't exist create it.

        Args:
            base_path (Path): The top folder
                (parent to the subdirectory).
        '''
        if not self.folder_path:
            sub_folder = base_path / self.folder_name
            if not sub_folder.exists():
                sub_folder.mkdir()
            self.folder_path = sub_folder

    def set_tag(self):
        '''If tag not given use uppercase of folder name.'''
        if not self.tag:
            self.tag = self.folder_name.upper()


## Build Boilerplate Files

### Build Jupyter lab Batch file

In [ ]:

jupter_batch_file = env_folder / f'JupyterLab ({new_env}).bat'
jupyter_batch = '\n'.join([
    r'CALL C:\ProgramData\Anaconda3\Scripts\activate.bat C:\ProgramData\Anaconda3',
    f'CALL conda activate "{new_env}"',
    f'CD "{project_folder}"',
    f'{project_drive}',
    'jupyter-lab'
    ])
jupter_batch_file.write_text(jupyter_batch)


### Build VS Code Batch file

In [ ]:
vscode_batch_file = env_folder / f'VS Code ({new_env}).bat'
vscode_batch = '\n'.join([
    r'CALL C:\ProgramData\Anaconda3\Scripts\activate.bat C:\ProgramData\Anaconda3',
    f'CALL conda activate "{new_env}"',
    f'CD "{project_folder}"',
    f'{project_drive}',
    f'code {new_env}.code-workspace'
    ])
vscode_batch_file.write_text(vscode_batch)


### Build .env file

In [ ]:
folders


In [ ]:
folders[folders.add_to_path]


In [ ]:
folders[folders.tag=='ENVIRONMENT'].folder_path


In [ ]:
env_file = env_folder / '.env'

env_paths = '\n'.join('PYTHONPATH=${}' for name, path_str in path_dict.items())
path_setter = '\n'.join('PYTHONPATH="${' + name.upper() + '};${PYTHONPATH}"'
                        for name in path_dict.keys())
env_text = '\n'.join([env_paths, path_setter])
env_file.write_text(env_text)


### Build Code-workspace from templatestr(env_path)

In [ ]:
data_dir = Path(r"D:\OneDrive - Queen's University\Python\Python Learning\Environment Management")

worspace_template = data_dir / 'vscode settings templates' / 'TEMPLATE.code-workspace'


In [ ]:
worspace = worspace_template.read_text()


In [ ]:
path_dict = {
    'WORKSPACE_FOLDER': f'"{project_folder}"',
    'SOURCE_FOLDER': f'"{source_folder}"',
    'SECTIONARY_FOLDER': f'"{sectionary_path}"'
    }


In [ ]:
str(env_path).replace('\\', '/')


In [ ]:
env_manage_base = Path(r"D:\OneDrive - Queen's University\Python\Python Learning\Environment Management")
code_workspace_template = env_manage_base / 'vscode settings templates' / 'TEMPLATE.code-workspace'


In [ ]:
template = code_workspace_template.read_text()


In [ ]:
template = template.replace('%env_path%', str(env_path).replace('\\', '/'))


In [ ]:
project_folder = Path(r"D:\OneDrive - Queen's University\Python\Projects\ToDoIst Projects\Turkey")

env_folder = add_folder(project_folder, 'environment')
source_folder = add_folder(project_folder, 'src')

project_drive = project_folder.drive


In [ ]:
sectionary_path = project_folder / '..' / '..' / 'sectionary package' / 'src'
sectionary_path = sectionary_path.resolve()


In [ ]:
path_dict = {
    'WORKSPACE_FOLDER': f'"{project_folder}"',
    'SOURCE_FOLDER': f'"{source_folder}"',
    'SECTIONARY_FOLDER': f'"{sectionary_path}"'
    }


In [ ]:
template.replace('%analysis.extraPaths%', str(text))


In [ ]:
template.replace('%autoComplete.extraPaths%', str(text))


## Other install commands

#### Install Jupyter Kernel 

In [ ]:
def install_kernel(new_env):
    '''Install the ipython kernel in the new environment.

    Args:
        new_env (str): Name of Conda environment.

    Returns:
        str: The log output from the install.
    '''
    kernel_cmd = activate_environment(new_env)
    kernel_cmd += f'python -m ipykernel install --user --name {new_env} '
    kernel_cmd += f'--display-name "Python ({new_env})"'

    kernel_install_log = console_command(
        kernel_cmd,
        AnacondaException,
        f'Unable to install ipython kernel in "{new_env}"!'
        )
    return kernel_install_log


#### Configure xlwings

In [ ]:
def configure_xlwings(new_env: str)->str:
    '''Install and configure xlwings binaries for environment.

    NOTE: This is still a works in progress.

    Args:
        new_env (str): Name of Conda environment.

    Returns:
        str: The log output from the install.
    '''
    activate_cmd = activate_environment(new_env)

    logger.debug('Updating Xlwings licence for %s', new_env)
    xlwings_license = activate_cmd + 'xlwings license update -k noncommercial'
    license_log = console_command(
        xlwings_license, AnacondaException,
        f'Unable to set xlwings license in environment {new_env}')

    logger.debug('Installing Xlwings add-in for %s', new_env)
    xlwings_add_in = activate_cmd + 'xlwings addin install'
    try:
        add_in_log = console_command(
            xlwings_add_in, AnacondaException,
            f'Unable to install xlwings addin in environment {new_env}',
            abort_after=2)
    except AbortedCmdException:
        logger.warning('Xlwings Add-in install timed out\n'
                       'This is likely due to insufficient rights to install '
                       'the add-in.')

    logger.debug('Creating Xlwings config file for %s', new_env)
    xlwings_config = activate_cmd + 'xlwings config create --force'
    config_log = console_command(
        xlwings_config, AnacondaException,
        f'Unable to configure xlwings in environment {new_env}')

    xlwings_log = '\n\n'.join([license_log, add_in_log, config_log])
    return xlwings_log


***The commands below hang the kernel***

In [ ]:
new_env = 'Todoist'
activate_cmd = activate_environment(new_env)

xlwings_license = activate_cmd + 'xlwings license update -k noncommercial'
license_log = console_command(
    xlwings_license, AnacondaException,
    f'Unable to set xlwings license in environment {new_env}')

print(license_log)


In [ ]:
cmd_str = activate_environment('Standard')
cmd_str += 'conda env export --from-history'
output = console_command(cmd_str)
print(output)


In [ ]:
output = install_kernel('Todoist')
print(output)


## Store Environment Data

In [ ]:
# FIXME Complete functions on saving log data and environment info


**This is a lower priority work in progress.**

### Write the Log Data generated during the Conda Environment install

**TODO:**
1. Generate Json file with output logs
2. generate a spreadsheet with multiple tables containing relevant project info.
   1. Package list:
      1. Package parsing must allow for 1, 2, or 3 parts.
      2. Package parsing must allow for '=' or '==' separators.
      3. If "from-history" used, pip installs are not returned; need to get those separately.
   2. Install parameters
      1. Name, prefix, channels
      2. List of channels should be converted to strings.
   3. Relevant folders and paths
      1. Project folders
      2. python search paths
      3. Other tools installed e.g. Jupyter, Xlwings


Get package Info has two different functionalities:
   1. Generate spreadsheet with package info
   2. Provide package lookup

In [ ]:
project_folder = Path(r"D:\OneDrive - Queen's University\Python\Projects\TestProject")
env_folder = add_folder(project_folder, 'environment')


#### Store json log output as a .json file

In [ ]:
def save_logs(create_output, install_output, pip_install_output, env_name, env_folder):
    environment_logs = {'Create': create_output,
                        'Install': install_output,
                        'Pip Install':pip_install_output}
    log_file = env_folder / f'{env_name}EnvironmentLog.json'
    with log_file.open('w', encoding='utf-8') as file:
        json.dump(environment_logs, file, indent=4)


#### Create a spreadsheet with environment data for reference

In [ ]:
env_ref = env_path


In [ ]:
env_name, env_cmd_ref = set_env_ref(env_ref)


In [ ]:
def build_package_table(dependencies):
    def  build_pip_package_list(pkg_ref):
        pip_pkg_list = []
        pip_pkg_info = pkg_ref['pip']
        for pkg_str in pip_pkg_info:
            name, version = pkg_str.split('==')
            pip_pkg_list.append({
                            'PackageName': name,
                            'PackageVersion': version,
                            'PIP': True
                            })
        return pip_pkg_list

    packages = []
    for pkg_str in dependencies:
        if isinstance(pkg_str,dict):
            pip_pkg_list = build_pip_package_list(pkg_str)
            packages.extend(pip_pkg_list)
        else:
            name, version, ref = pkg_str.split('=')
            packages.append({
                'PackageName': name,
                'PackageVersion': version,
                'PackageIndex': ref,
                'PIP': False
                })
    package_table = pd.DataFrame(packages)
    return package_table


In [ ]:
package_cmd = 'conda env export --json '
package_cmd += env_cmd_ref

pkg_output = console_command(package_cmd, AnacondaException,
                f'Error getting packages from {env_name}.')

pkg_dict = json.loads(pkg_output)

pkg_list = pkg_dict['dependencies']
packages = []
for pkg_str in pkg_list:
    if isinstance(pkg_str,dict):
        # Put pip package parser here
        pass
    else:
        name, version, ref = pkg_str.split('=')
        packages.append({
            'PackageName': name,
            'PackageVersion': version,
            'PackageIndex': ref,
            'PIP': False
            })
package_table = pd.DataFrame(packages)
package_table

dependencies = pkg_dict.pop('dependencies')
build_package_table(dependencies)


In [ ]:
pkg_dict['channels'] = str(pkg_dict['channels'])


In [ ]:
def get_packages(env_ref: EnvRef, all_packages=False)->pd.DataFrame:
    env_name, env_cmd_ref = set_env_ref(env_ref)
    if all_packages:
        package_cmd = 'conda env export --json'
        package_cmd += env_cmd_ref

        save_history_cmd = ['conda', 'env', 'export',
                            '--from-history', '--json']
        save_history_cmd += env_cmd_ref
        save_history_cmd += ['>', str(history_json)]
        console_command(save_history_cmd, AnacondaException,
                        f'Error saving {env_name}.json file.')


In [ ]:

log_file = env_folder / f'{new_env}EnvironmentLog.json'
pkg_table_file = env_folder / f'{new_env}Packages.csv'

install_log = install_log + '\n' + pkg_install_log
log_file.write_text(install_log)

pkg_items = logging_dict['actions']['LINK'] + pkg_install_dict['actions']['LINK']
pkg_table = pd.DataFrame(pkg_items)
pkg_table.to_csv(pkg_table_file)


# Misc Reference

## cmd
Starts a new instance of the command interpreter, Cmd.exe. If used without parameters, cmd displays the version and copyright information of the operating system.

#### Syntax
`cmd [/c|/k] [/s] [/q] [/d] [/a|/u] [/t:{<b><f> | <f>}] [/e:{on | off}] [/f:{on | off}] [/v:{on | off}] [<string>]`

#### Parameters

|Parameter|Description|
|-|-|
|/c|Carries out the command specified by <string> and then exits the command processor.|
|/k|Carries out the command specified by <string> and keeps the command processor running.|
|/s|When used with /c or /k, triggers special non-parsing rules that strip the first and last quotes (") around the <string> but leaves the rest of the command unchanged.|
|/q|Turns echo off.|
|/d|Disables execution of AutoRun commands.|
|/a|Formats command output as American National Standards Institute (ANSI).|
|/u|Formats command output as Unicode.|
|/t: {\<b>\<f> \| \<f>}|Sets the background (*b*) and foreground (*f*) colors.|
|/e:on|Enables command extensions.|
|/e:off|Disables commands extensions.|
|/f:on|Enables file and directory name completion.|
|/f:off|Disables file and directory name completion.|
|/v:on|Enables delayed environment variable expansion.|
|/v:off|Disables delayed environment variable expansion.|
|\<string>|Specifies the command you want to carry out.|
|/?|Displays help at the command prompt.|

The following table lists valid hexadecimal digits that you can use as the values for \<b> and \<f>:

|Value|Color|
|-|-|
|0|Black|
|1|Blue|
|2|Green|
|3|Aqua|
|4|Red|
|5|Purple|
|6|Yellow|
|7|White|
|8|Gray|
|9|Light blue|
|a|Light green|
|b|Light aqua|
|c|Light red|
|d|Light purple|
|e|Light yellow|
|f|Bright white

#### Remarks
- To redirect command output to the input of another command, use the pipe (`|`) character. For example:
> `<command1> | <command2>`

- To redirect command output to a file, use the greater-than angle bracket `>` character. For example:

`<command1> > <file1.txt>`
To use multiple commands for `<string>`, separate them by the command separator `&&`. For example:
> `<command1>&&<command2>&&<command3>`

- If the directory path, files, or any information you supply contains spaces, you must use double quotation marks (`"``) around the text, such as `"Computer Name"`. For example:
> `mkdir Test&&mkdir "Test 2"&&move "Test 2" Test`
> 
- You must use quotation marks around the following special characters: & \< > [ ] | { } ^ = ; ! ' + , ` ~ [white space].

- If you specify `/c` or `/k`, the `cmd` processes, the remainder of `<string>`, and the quotation marks are preserved only if all of the following conditions are met:
   - You don't also use `/s`.
   - You use exactly one set of quotation marks.
   - You don't use any special characters within the quotation marks (for example: & <    ( ) @ ^ | ).
   - You use one or more white-space characters within the quotation marks.
   - The `<string>` within quotation marks is the name of an executable file.

If the previous conditions aren't met, `<string>` is processed by examining the first character to verify whether it's an opening quotation mark. If the first character is an opening quotation mark, it's stripped along with the closing quotation mark. Any text following the closing quotation marks is preserved.

## Common Packages
> **Primary Packages**
> - numpy
> - pandas
> - openpyxl
> - matplotlib
> - seaborn
> - pysimplegui

> **iPython Packages**
> - ipykernel
> - ipywidgets
> - jupyterlab
> - nb_conda_kernels
> - spyder

> **SciPy Packages**
> - scipy
> - scikit-image
> 
> **Specialty Packages**
> - networkx
> - pygraphviz
> - shapely
> - pydicom
> - pyodbc
> - parsedatetime

> **MS Office & Windows related Packages**
> - xlwings
> - pywin32
> - docx2txt
> - zipfile2

> **Linting Packages**
> - pylint
> - mypy
> - black

> **Installer**
> - pyinstaller

> **Documentation Packages**
> - sphinx
> - sphinx-copybutton
> - sphinx_rtd_theme
> - nbsphinx-link
> - nbsphinx
> - esbonio
> - pandoc
> - myst-parser

> **AI Related Packages**
> - scikit-learn
> - neptune-client
> - neptune-sklearn
> - statsmodels
> - xgboost
> - torchvision
> - nibabel
> - nilearn


### Other Install Commands

`pip install todoist-api-python`

```bat
pip install [options] <requirement specifier> [package-index-options] ...
pip install [options] -r <requirements file> [package-index-options] ...
pip install [options] [-e] <vcs project url> ...
pip install [options] [-e] <local project path> ...
pip install [options] <archive url/path> ...
```
The report can be written to a file, or to standard output (using --report - in combination with --quiet).

#### Json Basic Usage
`json.dump(obj, fp, *, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw)`

Serialize obj as a JSON formatted stream to fp (a .write()-supporting file-like object) using this conversion table.

- If skipkeys is true (default: False), then dict keys that are not of a basic type (str, int, float, bool, None) will be skipped instead of raising a TypeError.

- The json module always produces str objects, not bytes objects. Therefore, fp.write() must support str input.

- If ensure_ascii is true (the default), the output is guaranteed to have all incoming non-ASCII characters escaped. If ensure_ascii is false, these characters will be output as-is.

- If check_circular is false (default: True), then the circular reference check for container types will be skipped and a circular reference will result in a RecursionError (or worse).

- If allow_nan is false (default: True), then it will be a ValueError to serialize out of range float values (nan, inf, -inf) in strict compliance of the JSON specification. If allow_nan is true, their JavaScript equivalents (NaN, Infinity, -Infinity) will be used.

- If indent is a non-negative integer or string, then JSON array elements and object members will be pretty-printed with that indent level. An indent level of 0, negative, or "" will only insert newlines. None (the default) selects the most compact representation. Using a positive integer indent indents that many spaces per level. If indent is a string (such as "\t"), that string is used to indent each level.

- If specified, separators should be an (item_separator, key_separator) tuple. The default is (', ', ': ') if indent is None and (',', ': ') otherwise. To get the most compact JSON representation, you should specify (',', ':') to eliminate whitespace.

- If specified, default should be a function that gets called for objects that can’t otherwise be serialized. It should return a JSON encodable version of the object or raise a TypeError. If not specified, TypeError is raised.

- If sort_keys is true (default: False), then the output of dictionaries will be sorted by key.

`pip freeze > requirements.txt`